In [1]:
# ============================================================
# 🧠 Notebook: Entrenamiento de Modelo YOLOv8 Pose - Salmo Metrics
# ============================================================
# Proyecto: Estimación automática de dimensiones de salmones
# Autor: Alonso Castillo
# Fecha: Octubre 2025
# ------------------------------------------------------------
# Este notebook entrena un modelo YOLOv8 pose (keypoints)
# utilizando un dataset exportado desde CVAT en formato COCO Keypoints.
# ============================================================

In [1]:
#librerías ---
import json
import os
from pathlib import Path
from tqdm import tqdm
import cv2
import numpy as np

from ultralytics import YOLO
import torch
from IPython.display import Image, display



In [3]:

# Convierte anotaciones de segmentación en formato COCO JSON al formato YOLOv8-Seg.
# una carpeta 'labels' paralela.
# Ejemplo de uso:
# Asegurarse que las carpetas se encuentren así:
#  salmon_segmentation_dataset/
#   - images/
#     - train/
#       - img1.jpg
#       -...
#     - val/
#       - imgN.jpg
#       -...
#   - annotations/
#     - instances_train.json
#     - instances_val.json

# convert_coco_to_yolo_seg('/salmon_segmentation_dataset/annotations/instances_train.json', '/segmentation_yolo_dataset/train/')
# convert_coco_to_yolo_seg('/salmon_segmentation_dataset/annotations/instances_val.json', '/segmentation_yolo_dataset/val/')

def convert_coco_to_yolo_seg(coco_json_path, save_dir):

    save_path = Path(save_dir)
    labels_path = save_path / 'labels'
    images_path = save_path / 'images'
    labels_path.mkdir(parents=True, exist_ok=True)

    with open(coco_json_path, 'r') as f:
        data = json.load(data)

    # Crear un mapeo de image_id a nombre de archivo y dimensiones
    images_info = {img['id']: {'file_name': img['file_name'], 'height': img['height'], 'width': img['width']} for img in data['images']}
    
    # Crear un mapeo de category_id a class_index (asumiendo que los IDs de COCO no son necesariamente 0-indexados)
    categories = {cat['id']: i for i, cat in enumerate(data['categories'])}

    # Agrupar anotaciones por image_id
    annotations_by_image = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
            annotations_by_image[img_id].append(ann)

    print("Iniciando conversión de anotaciones...")
    for img_id, anns in tqdm(annotations_by_image.items()):
        img_info = images_info[img_id]
        img_h, img_w = img_info['height'], img_info['width']
        
        # Validar que las dimensiones de la imagen son válidas
        if img_h == 0 or img_w == 0:
            print(f"Advertencia: La imagen {img_info['file_name']} tiene dimensiones cero. Se omitirá.")
            continue

        label_name = Path(img_info['file_name']).stem + '.txt'
        label_path = labels_path / label_name

        with open(label_path, 'w') as f:
            for ann in anns:
                # Validar que es una anotación de segmentación
                if 'segmentation' not in ann or not ann['segmentation']:
                    continue

                class_index = categories[ann['category_id']]
                
                # El formato de segmentación de COCO puede tener múltiples polígonos para una instancia
                for seg in ann['segmentation']:
                    # Validar que el polígono tiene al menos 3 vértices
                    if len(seg) < 6:
                        print(f"Advertencia: Polígono inválido (menos de 3 vértices) en {img_info['file_name']}. Se omitirá.")
                        continue
                    
                    # Normalizar coordenadas
                    normalized_coords = [] 
                    for i in range(0, len(seg), 2):
                        x = seg[i] / img_w
                        y = seg[i+1] / img_h
                        normalized_coords.extend([x, y])
                    
                    # Escribir en el archivo de etiqueta
                    line = f"{class_index} " + " ".join(map(str, normalized_coords))
                    f.write(line + '\n')

    print(f"Conversión completada. Las etiquetas se guardaron en: {labels_path}")


### Procesado de datos (normalizacion)

In [ ]:
 convert_coco_to_yolo_seg('/salmon_segmentation_dataset/annotations/instances_train.json', '/segmentation_yolo_dataset/train/')
 convert_coco_to_yolo_seg('/salmon_segmentation_dataset/annotations/instances_val.json', '/segmentation_yolo_dataset/val/')


## Cargar un modelo de segmentación pre-entrenado YOLOv8n-seg.pt

In [4]:

# 'n' se refiere al modelo nano, el más pequeño y rápido. 
# Se pueden usar otros tamaños como 's', 'm', 'l', 'x' para mayor precisión a costa de velocidad.
model = YOLO('yolov8n-seg.pt')  # Carga pesos pre-entrenados


## Entrenar el modelo con el conjunto de datos de salmones

In [ ]:

print("Iniciando el entrenamiento del modelo de segmentación...")
results = model.train(
    data='salmon_segmentation_dataset/data.yaml',
    epochs=100,          # Número de épocas de entrenamiento
    imgsz=640,           # Tamaño de imagen de entrada
    batch=16,            # Tamaño del lote
    patience=20,         # Épocas a esperar sin mejora antes de detener el entrenamiento
    name='yolov8n_salmon_seg', # Nombre del experimento
    # Parámetros de aumento de datos (data augmentation)
    # Cruciales para la robustez en entornos subacuáticos
    hsv_h=0.015,  # Aumento de matriz
    hsv_s=0.7,    # Aumento de saturación
    hsv_v=0.4,    # Aumento de valor (brillo)
    degrees=0.0,  # Rotación
    translate=0.1,# Traslación
    scale=0.5,    # Escala (zoom)
    flipud=0.5,   # Flip vertical
    mosaic=1.0,   # Aumento de mosaico
    mixup=0.1     # Aumento de mixup
)
print("Entrenamiento completado.")
print(f"El mejor modelo se guardó en: {results.save_dir}")